<a href="https://colab.research.google.com/github/MauriLoria/Python/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
ruta = "/content/ARTICULO.DBF"

Saving ARTICULO.DBF to ARTICULO.DBF


In [2]:
%pip install dbfread

In [3]:
from dbfread import DBF

# Se utiliza la funcion try para el manejo de errores.
try:
    table = DBF(ruta, encoding='latin-1')
    df = pd.DataFrame(list(table))
    display(df.head())
except UnicodeDecodeError:
    print("UnicodeDecodeError: Error en la codificación del archivo.")
except Exception as e:
    print(f"Se produjo un error: {e}")

,CODIGO,DESCRIP,UNIDAD,VAL_UNID,FAMILIA,STOCKACT,STOCKMIN,PRECIOACT,PRECIOCOM,PRECIOMAY,...,PROV3,COSTO3,BONIFCANT,IVA,DOLAR,CTACTE,CANTMAY,IVAESP,GANANMAY,SCAN
0,00001006,"REDONDO 1010 1/4"" 0,265 x MTS",KG,1,000,1521.85,45.00,3375.66,1690.79,2282.57,...,IVANAR,9.93,0.82,21.0,N,17.36,1.6,T,1.35,
1,00001008,"REDONDO 1010 5/16"" 0,400 x MTS",KG,1,000,3189.30,24.00,3910.03,1958.44,2643.89,...,PERKUSIC,0.00,0.82,21.0,N,17.36,2.4,T,1.35,
2,00001010,"REDONDO 1010 3/8"" 0,600 x MTS",KG,1,000,3903.31,108.00,3834.94,1920.83,2593.12,...,IVANAR,8.31,0.82,21.0,N,17.36,3.6,T,1.35,
3,00001011,"REDONDO 1010 7/16"" 0,780 x MTS",KG,1,000,936.12,45.75,3767.38,1886.99,2547.44,...,IVANAR,5.35,0.82,21.0,N,17.36,4.6,T,1.35,
4,00001013,"REDONDO 1010 1/2"" 1,050 x MTS",KG,1,000,4439.71,360.00,3478.72,1742.41,2352.25,...,NAVARRO,8881.36,0.82,21.0,N,17.36,6.3,T,1.35,


In [4]:
# Se Muestran las columnas.
df.columns

Index(['CODIGO', 'DESCRIP', 'UNIDAD', 'VAL_UNID', 'FAMILIA', 'STOCKACT',
       'STOCKMIN', 'PRECIOACT', 'PRECIOCOM', 'PRECIOMAY', 'PREVTA01',
       'PREVTA02', 'PREVTA03', 'PREVTA04', 'INDICE', 'TIPIND', 'GASTOS',
       'GANANCIA', 'CANTVTA', 'CANTMES', 'IMPCONTA', 'IMPCPRA', 'NROCODOR',
       'TILDE', 'ALMACEN0', 'PROV', 'COSTO', 'PROV1', 'COSTO1', 'PROV2',
       'COSTO2', 'PROV3', 'COSTO3', 'BONIFCANT', 'IVA', 'DOLAR', 'CTACTE',
       'CANTMAY', 'IVAESP', 'GANANMAY', 'SCAN'],
      dtype='object')

In [5]:
# Se realiza un filtrado para utilizar las columnas necesarias.
arti_stk = df[["CODIGO", "DESCRIP", "FAMILIA", "STOCKACT", "STOCKMIN"]]

In [6]:
# Se mostran los primeros cinco movimientos para ver el nuevo Dataframe.
arti_stk.head()

,CODIGO,DESCRIP,FAMILIA,STOCKACT,STOCKMIN
0,00001006,"REDONDO 1010 1/4"" 0,265 x MTS",000,1521.85,45.00
1,00001008,"REDONDO 1010 5/16"" 0,400 x MTS",000,3189.30,24.00
2,00001010,"REDONDO 1010 3/8"" 0,600 x MTS",000,3903.31,108.00
3,00001011,"REDONDO 1010 7/16"" 0,780 x MTS",000,936.12,45.75
4,00001013,"REDONDO 1010 1/2"" 1,050 x MTS",000,4439.71,360.00


In [7]:
# Se carga el archivo con los movimientos de stock.
from google.colab import files
uploaded = files.upload()

Saving MOVDISTK.DBF to MOVDISTK.DBF


In [8]:
# Se asigna el archivo a un Dataframe
ruta = "/content/MOVDISTK.DBF"
table = DBF(ruta, encoding='latin-1')
movdi_AFA = pd.DataFrame(list(table))

In [9]:
movdi_AFA = movdi_AFA[["CODIGO", "CANTIDAD"]] # Se filtra para utilizar las columnas necesarias.
movstk = movdi_AFA.groupby("CODIGO")["CANTIDAD"].sum().reset_index()  # Se utiliza groupby para asignar la suma de los movimientos a cada artículo.
merged_df = arti_stk.merge(movstk, on='CODIGO', how='left', sort=True) # Se combina el archivo de artículos y el de los movimientos.
merged_df.fillna(0, inplace=True) # Se convierten los datos no numéricos en 0 (cero)

In [10]:
from google.colab import data_table # Se importa la función para mostrar los datos en una tabla interactiva.

In [11]:
merged_df["STOCKSACT"] = merged_df["STOCKACT"] + merged_df["CANTIDAD"]  # Se crea una columna que contiene el stock actual y se suman los movimientos.
merged_df.drop(columns=["STOCKACT", "CANTIDAD"], inplace=True)  # Se eliminan las columnas innecesarias.
data_table.DataTable(merged_df, include_index=False, num_rows_per_page=10)  # Se muestra la tabla interactiva.


,CODIGO,DESCRIP,FAMILIA,STOCKMIN,STOCKSACT
0,00000000,RECORTES,000,0.0,-6074.39
1,00000001,TORTAS DE CHAPA * KG,000,0.0,0.00
2,00001006,"REDONDO 1010 1/4"" 0,265 x MTS",000,45.0,1537.75
3,00001008,"REDONDO 1010 5/16"" 0,400 x MTS",000,24.0,3201.30
4,00001010,"REDONDO 1010 3/8"" 0,600 x MTS",000,108.0,3903.31
...,...,...,...,...,...
12857,WEBPER01,PASTINA WEBER PERLA 1 KG,902,0.0,20.00
12858,WEBPLA01,PASTINA WEBER PLATA * KG,902,0.0,20.00
12859,WEBTER01,PASTINA WEBER TERRACOTA * KG,902,0.0,10.00
12860,YARLDF05,LINTERNA 41 LEDS BAILONG POLICE,519,0.0,2.00


In [12]:
codigo = input("Ingrese Código a consultar: ")  # Se solicita el ingreso de un código para la consulta.
stk_ind = merged_df[merged_df.iloc[:,0] ==codigo.upper()] # Se filtra el Dataframe en busqueda de el artículo ingresado.
if not stk_ind.empty: # Se verifica si el Dataframe filtrado no esté vacío.
  print(stk_ind[["CODIGO", "DESCRIP", "STOCKSACT", "STOCKMIN"]].to_string(index=False, justify='center')) # Se convierte en string, alinea las columnas y se visualiza el resultado.
else:
  print("El código ingresado no figura en la base de datos.") # Se visualiza el error si no encontró el código ingresado.

Ingrese Código a consultar: 046orc10
 CODIGO                  DESCRIP                  STOCKSACT  STOCKMIN
046ORC10 MEMBRANA ORMIFLEX 4MM CODIGO 10 (46 KG)  -153.55      10.0  
